# Profitability vs Sentiment Analysis

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Set style and color palette
plt.style.use('seaborn-darkgrid')
sns.set_palette('husl')

# Read the merged data
df = pd.read_csv('merged_data.csv')

# Display first few rows
df.head()

## Calculate Daily Profitability

In [ ]:
# Convert timestamp to datetime
df['Date'] = pd.to_datetime(df['Timestamp'])

# Calculate daily PnL
daily_pnl = df.groupby('Date')['PnL'].sum().reset_index()

# Get daily sentiment values
daily_sentiment = df.groupby('Date')['value'].first().reset_index()

# Merge daily PnL with sentiment
daily_analysis = pd.merge(daily_pnl, daily_sentiment, on='Date')

# Display first few rows of the merged data
daily_analysis.head()

## Visualize Profitability vs Sentiment

In [ ]:
# Create enhanced scatter plot
plt.figure(figsize=(12, 6))

# Create main scatter plot with alpha for better visibility
scatter = plt.scatter(daily_analysis['value'], daily_analysis['PnL'],
                     c=daily_analysis['PnL'], cmap='coolwarm',
                     alpha=0.6, s=100)

# Add trend line with confidence interval
sns.regplot(data=daily_analysis, x='value', y='PnL',
           scatter=False, color='black', line_kws={'linestyle':'--'})

# Customize plot
plt.title('Daily Profitability vs Fear & Greed Index', fontsize=14, pad=20)
plt.xlabel('Fear & Greed Index', fontsize=12)
plt.ylabel('Daily PnL', fontsize=12)

# Add colorbar
plt.colorbar(scatter, label='PnL Value')

# Add grid for better readability
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Calculate Correlation

In [ ]:
# Calculate correlation between PnL and Fear & Greed Index
correlation = daily_analysis['PnL'].corr(daily_analysis['value'])

# Create a styled correlation heatmap
corr_matrix = daily_analysis[['PnL', 'value']].corr()

plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, cmap='RdYlBu', center=0,
            square=True, fmt='.2f', cbar_kws={'label': 'Correlation Coefficient'})
plt.title('Correlation Heatmap: PnL vs Fear & Greed Index', pad=20)
plt.tight_layout()
plt.show()

print(f'
Correlation between Daily PnL and Fear & Greed Index: {correlation:.2f}')

## Analyze Profitability by Sentiment Categories

In [ ]:
# Create sentiment categories
daily_analysis['Sentiment_Category'] = pd.cut(daily_analysis['value'],
                                             bins=[0, 25, 50, 75, 100],
                                             labels=['Extreme Fear', 'Fear', 'Neutral', 'Greed'])

# Create enhanced box plot
plt.figure(figsize=(12, 6))

# Create violin plot with box plot inside
sns.violinplot(data=daily_analysis, x='Sentiment_Category', y='PnL',
               palette='RdYlBu', inner='box')

# Customize plot
plt.title('Daily PnL Distribution by Market Sentiment', fontsize=14, pad=20)
plt.xlabel('Market Sentiment', fontsize=12)
plt.ylabel('Daily PnL', fontsize=12)
plt.xticks(rotation=45)

# Add grid for better readability
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Calculate and display statistics
stats_by_sentiment = daily_analysis.groupby('Sentiment_Category').agg({
    'PnL': ['mean', 'std', 'count']
}).round(2)

print('
Detailed Statistics by Sentiment Category:')
print(stats_by_sentiment)